In [0]:
import os
import pandas as pd
import numpy as np
os.chdir('/content/drive/My Drive/tabditor/FromZero/rsc')

# 데이터 준비

In [3]:
label = pd.read_csv('label_1.csv')
print(label.shape)
label.head()

(5541, 7)


,index,qus,ans,prep_qus,label,prepro_label,prepro_ans
0,9269,[Olive Young과 함께 마음까지 All live young하는 삶을 꿈꾸다],\n제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의...,OliveYoung과함께마음까지Allliveyoung하는삶을꿈꾸다,[1],1,제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의 방...
1,17155,경쟁과 협력 가운데 더 중요하다고 생각하는 것,\r\n무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다. 함께 ...,경쟁과협력가운데더중요하다고생각하는것,[1],1,무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다 함께 힘을 합친...
2,9623,[고퀄리티 제품과 서비스를 갖춘 GS엠비즈의 고퀄리티 영업관리자가 되겠습니다],"\r\nGS엠비즈는 ""The Best Company for Customers""를 비...",고퀄리티제품과서비스를갖춘GS엠비즈의고퀄리티영업관리자가되겠습니다,[1],1,GS엠비즈는 The Best Company for Customers를 비전으로 자동...
3,4410,공공기관으로서 사업 추진에 필요한 영화계 의견수렴 방법을 제시하여 주십시오 1000바이트,"\r\n""현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성""\r 저는 영화진...",공공기관으로서사업추진에필요한영화계의견수렴방법을제시하여주십시오1000바이트,"[1,2]","1,2",현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성 저는 영화진흥위원회의 의견...
4,7456,공공기관으로써 갖추어야 할 덕목,\r\n한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각...,공공기관으로써갖추어야할덕목,[1],1,한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각하는 배...


# text preprocessing

In [4]:
label.prepro_ans

0       제가 올리브영에 입사하고 싶은 이유는 올리브영만의 기업철학이 제가 지향하는 삶의 방...
1       무한경쟁시대일수록 역설적으로 협력의 힘이 더욱 중요하다고 생각합니다 함께 힘을 합친...
2       GS엠비즈는 The Best Company for Customers를 비전으로 자동...
3       현장 상황을 존중하는 적극적인 자세와 직접 소통의 중요성 저는 영화진흥위원회의 의견...
4       한국토지공사는 사기업과는 다른 공기업이기 때문에 개인보다는 국민을 먼저 생각하는 배...
                              ...                        
5536    식품 연구원으로의 성장저의 지도교수님은 한 분야의 전문가가 되려면 최소 10년의 지...
5537    실험을 통한 분석력과 조별과제에서 배운 소통능력 생산 관리 직무는 현장에서 생산 계...
5538    저는  태권도 동아리 소속의 아낌없이 가르쳐주고 소통하는 멘토입니다에 입학 후 건강...
5539    병원에 입원해 본 사람은 매 시간이 얼마나 감사한지 알 것입니다 사고로 군용 차량과...
5540    병원에 입원해 본 사람은 매 시간이 얼마나 감사한지 알 것입니다 사고로 군용 차량과...
Name: prepro_ans, Length: 5541, dtype: object

In [0]:
from keras.preprocessing.text import Tokenizer

token = Tokenizer()
token.fit_on_texts(label.prepro_ans)

In [22]:
len(token.word_index)

124385

In [35]:
token.index_word[472]

'본'

In [0]:
slide_data = []

for i in label.prepro_ans:
    seq_data = token.texts_to_sequences([i])
    for j in range(1,len(seq_data[0])):
        tmp = seq_data[0][0:j+1]
        slide_data.append(tmp)

        

In [48]:
len(slide_sen_data)

5541

In [50]:
maxlen = max([len(i) for i in slide_sen_data])
maxlen

785537

In [0]:
from keras.preprocessing.sequence import pad_sequences

seq_data = pad_sequences(slide_sen_data,maxlen=maxlen)
seq_data[:10]